In [1]:
%cd YOUR_PROJECT_DIRECTORY

/media/hafiz031/Windows/Users/Hafiz/Google_Drive_hafiz031demo@gmail_com/Thesis42/hafiz031/Extra_work000/NLP/Selenium_py


In [171]:
import pandas as pd
import numpy as np

In [172]:
# Enter the CSV file directory here which contains the comments
df = pd.read_csv("")
df.head()

,username,comment,timestamp,like_count
0,Balaj Prasanna,"Well I thought it was 10% luck, 20% skill, 15%...",1 week ago,0
1,Sadmim Hossain,"There is a saying in Bangladesh that, ""if it i...",1 day ago,99
2,Nabiel Adrian,"“Fate leads the willing, and drags along the r...",3 days ago,94
3,Craig Kinsey,This reminds me of the ancient principle that ...,1 day ago,5
4,Vibhav G,I “like him” as he acknowledged that luck got ...,3 days ago,9


In [173]:
# removing rows having missing values
df = df.dropna()

Separating out emojis and texts such that we can process them individually

In [174]:
!pip install emoji

import emoji

def emoji_extractor(text):
  return ''.join(e for e in text if e in emoji.UNICODE_EMOJI)

In [175]:
emojis = df['comment'].apply(emoji_extractor).values

# emojis (if any) picked out from each of the comments
emojis

array(['', '', '', ..., '', '', ''], dtype=object)

In [176]:
# now removing emojis (if any) from the corresponding comments
df.tail()

,username,comment,timestamp,like_count
1555,jaycenf29,Cara wants you to call her,5 days ago,0
1556,Jenny Castrejon,"the problem is procrastination , just do it.",5 days ago,0
1557,Lafemmebear Music,But white mens ego's and the world supporting ...,5 days ago,0
1558,Diogo Meneses,"In retrospect, being lucky is actually a great...",1 week ago,16
1559,Bryce Walburn,I wish more politicians were abducted by aliens,4 days ago,0


In [177]:
import re

def emoji_remover(string):
    # Emoji pattern from: https://stackoverflow.com/a/54313085/6907424
    emoji_pattern = re.compile("["
                    u"\U0001F600-\U0001F64F"  # emoticons
                    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                    u"\U0001F680-\U0001F6FF"  # transport & map symbols
                    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                    u"\U00002702-\U000027B0"
                    u"\U000024C2-\U0001F251"
                    u"\U0001f926-\U0001f937"
                    u'\U00010000-\U0010ffff'
                    u"\u200d"
                    u"\u2640-\u2642"
                    u"\u2600-\u2B55"
                    u"\u23cf"
                    u"\u23e9"
                    u"\u231a"
                    u"\u3030"
                    u"\ufe0f"
        "]+", flags=re.UNICODE)
    
    return emoji_pattern.sub(r'', string)

In [178]:
comments = df['comment'].apply(emoji_remover).values

In [179]:
# after removing emojis
comments

array(['Well I thought it was 10% luck, 20% skill, 15% concentrated power of will. 5% pleasure, 50% pain and a 100% reason to remember the name.13K',
       'There is a saying in Bangladesh that, "if it is the only hard work that pays off, then the farmers(/labors) would have been millionaires."',
       '“Fate leads the willing, and drags along the reluctant.”- Seneca',
       ...,
       "But white mens ego's and the world supporting those ego's think are always greater than their poc counterparts... thats just racism",
       'In retrospect, being lucky is actually a great qualification for being an astronaut',
       'I wish more politicians were abducted by aliens'], dtype=object)

# Testing

In [180]:
# to remove stop-words from the movie reviews, we will use the set of 127 English stop-words that is available from
# the NLTK library.

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [181]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [182]:
def tokenizer(text):
    return text.split()

In [183]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [184]:
# we will be using GridSearchCV to find the optimal set of parameters for our logistic regression model using
# 5-fold stratified cross-validation

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents = None,
                       lowercase = False,
                       preprocessor = None)

param_grid = [
    {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [stop, None],
        'vect__tokenizer': [tokenizer, tokenizer_porter],
        'clf__penalty': ['l1', 'l2'],
        'clf__C': [1.0, 10.0, 100.0]
    },
    
    {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [stop, None],
        'vect__tokenizer': [tokenizer, tokenizer_porter],
        'vect__use_idf': [False],
        'vect__norm': [None],
        'clf__penalty': ['l1', 'l2'],
        'clf__C': [1.0, 10.0, 100.0]
    }
]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state = 0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                          scoring = 'accuracy',
                          cv = 5, verbose = 1, 
                          n_jobs = -1)

Now loading the model for sentiment analysis and after fitting the test data and transforming them

In [185]:
import pickle

model = pickle.load(open("sentiment.model", "rb"))

In [186]:
verdicts = model.predict(comments)

sentiments = []

for verdict in verdicts:
    if verdict == 1:
        sentiments.append("Positive")
    else:
        sentiments.append("Negative")

In [187]:
sentiments

['Negative',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Negative',

In [188]:
# verdict on that video
print("[INFO] Analyzing the comments in this video: ")
verdict_sum = np.sum(verdicts)
positiveness = (verdict_sum / len(verdicts)) * 100
negativeness = 100 - positiveness
print("[INFO] Positiveness: ", positiveness, " and Negativeness: ", negativeness)

[INFO] Analyzing the comments in this video: 
[INFO] Positiveness:  53.58974358974359  and Negativeness:  46.41025641025641


Trying with single sentence (0 means negative and 1 means positive)

In [191]:
model.predict(['This is whats wrong with the gaming world today. I played this game 30 years ago. Sure it looked awful , but its the same game. Looks like it was more fun to make than to play. Which is 90% of games today. Devs have fallen up their own arses. And you know it wont come close to looking like that on release'])

array([0])

In [192]:
model.predict(['20 years ago this would ve blown my mind.And its still very impressive!'])

array([1])

In [222]:
index = 60
print(comments[index])
model.predict([comments[index]])

I've been trying to explain this for years to people, and NOW I have a good video I can point everyone to that gives a MUCH better summary than I have been able to.By the way, I love the CGP Grey shirt.


array([1])